In [4]:
import tensorflow as tf
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import pandas as pd
tf.__version__

Using TensorFlow backend


'2.14.0'

In [5]:
data = pd.read_csv("../input_data/CSVs/connect-4.csv")
data.head()

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,...,f4,f5,f6,g1,g2,g3,g4,g5,g6,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [6]:
target = data.pop('class')
target.head()

0    2
1    2
2    2
3    2
4    2
Name: class, dtype: int64

In [7]:
# Split the data into training and validation features and targets, with 20% being testing data
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [8]:
tf.random.set_seed(8)

# To use the keras tuner to automatically find optimal values to train the model, we need to define a function it can use to then build the model and insert the different values for it to test with
def model_builder(hp):
    model = tf.keras.Sequential()
    hp_l2 = hp.Choice('l2', values = [0.1, 0.01, 0.001, 0.0001])
    
    reg_fc1 = tf.keras.layers.Dense(512, input_shape=(42,), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
    reg_fc2 = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
    reg_fc3 = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
    reg_fc4 = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
    reg_fc5 = tf.keras.layers.Dense(3, activation='softmax')
    
    model.add(reg_fc1)
    model.add(reg_fc2)
    model.add(reg_fc3)
    model.add(reg_fc4)
    model.add(reg_fc5)

    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adam(0.001)
    model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])
    return model
    
# We can then use that function to tell it to use to test the values via various methods (this randomly selects values for example)
tuner = kt.RandomSearch(model_builder, objective='val_accuracy', max_trials=10)
tuner.search(X_train, y_train, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 15s]
val_accuracy: 0.6568975448608398

Best val_accuracy So Far: 0.7568827867507935
Total elapsed time: 00h 00m 58s


In [9]:
# We can then get the best parameters it found, and use those to set up our model in the most optimal way automatically
best_hps = tuner.get_best_hyperparameters()[0]

best_l2 = best_hps.get('l2')
# Show the best value for `l2` it found
best_l2

0.0001

In [10]:
# We can then finally build the model using the optimal hyperparameters found
model = tuner.hypermodel.build(best_hps)

model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
1689/1689 [==============================] - 13s 7ms/step - loss: 0.7390 - accuracy: 0.7164 - val_loss: 0.6485 - val_accuracy: 0.7515
Epoch 2/5
1689/1689 [==============================] - 12s 7ms/step - loss: 0.6043 - accuracy: 0.7730 - val_loss: 0.5939 - val_accuracy: 0.7777
Epoch 3/5
1689/1689 [==============================] - 12s 7ms/step - loss: 0.5509 - accuracy: 0.7947 - val_loss: 0.5402 - val_accuracy: 0.7997
Epoch 4/5
1689/1689 [==============================] - 12s 7ms/step - loss: 0.5198 - accuracy: 0.8092 - val_loss: 0.5683 - val_accuracy: 0.7902
Epoch 5/5
1689/1689 [==============================] - 12s 7ms/step - loss: 0.4972 - accuracy: 0.8184 - val_loss: 0.5291 - val_accuracy: 0.8096


This has made a huge improvement, increasing testing data accuracy from 60% (in 6.02 with dropout) to 80% by testing for the best parameters to use with the L2 regulariser. *(Note that from the book's example output, a non-optimised L2 regulariser would have output similar testing accuracy to the dropout model)*